In [1]:
import requests
import re

In [28]:
accepted_units = ['small', 'medium', 'big', 'medium-size',
                  'g', 'pound', 'pounds', 
                  'ounce', 'ounces', 'oz', 'oz.',
                  'l', 'ml',
                  'pint',
                  'jar', 'cup', 'cups', 'can', 'cans'
                  'package',
                  'spoon', 'spoons', 'tablespoon', 'tablespoons', 'teaspoon', 'teaspoons',
                  'Pinch', 'pinch', 'handful',
                  'clove', 'cloves',
                  'slice', 'slices',
                  'bunch'
                 ]

In [40]:
top_source = 'https://www.cookingchanneltv.com/recipes/a-z/p/p/12'
source_letter = 'P'

In [41]:
r = requests.get(top_source)

In [42]:
# r.content

In [43]:
content = str(r.content)

In [44]:
idx_start = content.find('<span class="o-Capsule__a-HeadlineText">'+source_letter+'</span>')
content = content[idx_start:]
idx_stop = content.find('<section class="o-Pagination ">')
content = content[:idx_stop]

In [45]:
recepies_url_list = []
for ln in content.split('href')[1:]:
    idx_end = ln.find('">')
    url = 'https:' + ln[2:idx_end]
    recepies_url_list.append(url)
    
len(recepies_url_list)

37

In [9]:
url = recepies_url_list[3]
print(url)
content = str(requests.get(url).content)

https://www.cookingchanneltv.com/recipes/debi-mazar-and-gabriele-corcos/pasta-alla-gricia-2105351


In [39]:
def getName(content):
    template = '<span class="o-AssetTitle__a-HeadlineText">'
    idx_start = content.find(template) + len(template)
    s = content[idx_start:idx_start+100]
    idx_stop = s.find('</span>')
    title = s[:idx_stop]
    if ':' in title:
        title = title[:title.find(':')]
    print('\t'+title)
    
    return title

In [38]:
def getLevel(content):
    try:
        template = '<span class="o-RecipeInfo__a-Headline">Level:</span>'
        idx_start = content.find(template) + len(template)
        s = content[idx_start:idx_start+100]
        out = re.search(r'">[A-Za-z]+</span', s)
        level = out.group(0)[2:-6]
    except: level = 'None'
    print('\t'+level)
    
    return level

In [36]:
def getNumberPeople(content):
    try:
        template = '<span class="o-RecipeInfo__a-Headline">Yield:</span>'
        idx_start = content.find(template) + len(template)
        s = content[idx_start:idx_start+100]
        out = re.search(r'">[0-9]+', s)
        Nppl = out.group(0)[2:]
    except: 
        Nppl = '-1'
    print('\t'+Nppl)
    
    return int(Nppl)

In [37]:
def getIngredients(content):
    template = '<p class="o-Ingredients__a-Ingredient">'
    idx_start = content.find(template) + len(template)
    s = content[idx_start:]
    # print(s)
    cut_template = '<h6 class="o-Ingredients__a-SubHeadline">'
    if cut_template in s:
        idx_stop = s.find(cut_template)
        s = s[:idx_stop]
    slist = s.split(template)
    # print(slist)
    
    ingredients_list = []
    
    for l in slist:
        i = l.find('</p>')
        aux = l[:i]
#         print(aux)
        if ',' in aux:
            aux = aux[:aux.find(',')]
        if '(' in aux:
            aux = aux[:aux.find('(')-1] + aux[aux.find(')')+1:]
        if '/' in aux:
            aux = aux[aux.find('/')+1:]
        if aux[-8:] == '\\xc2\\xa0':
            aux = aux[:-8]

        l_aux = aux.split(' ')        
        if len(l_aux)==2 and l_aux[0].isnumeric():
            name = l_aux[1].capitalize()
            amount = l_aux[0]
            units = ''
        elif len(l_aux) > 2 and (l_aux[1] in accepted_units):
            name = ' '.join(l_aux[2:])
            name.capitalize()
            amount = l_aux[0]
            units = l_aux[1]
        elif len(l_aux) > 1 and not l_aux[0].isnumeric() and not (l_aux[1] in accepted_units):
            name = ' '.join(l_aux)
            name.capitalize()
            amount = ''
            units = ''
        elif len(l_aux) > 2 and l_aux[0].isnumeric() and not (l_aux[1] in accepted_units):
            name = ' '.join(l_aux[1:])
            name.capitalize()
            amount = l_aux[0]
            units = ''
        elif len(l_aux) == 1:
            name = l_aux[0]
            amount = ''
            units = ''
        else:
            print(l_aux)
            name = 'NaN'
            amount = 'NaN'
            units = 'NaN'
            
        if name.startswith('of '):
            name = name[3:]
        
        out = '\t\t'+ name + '-->'
        out += 'Amount: '+ amount + '(' + units + ')'
        print(out)
        
        ingredients_list.append([name, amount, units])
        
    return ingredients_list

In [25]:
# content

In [26]:
for url in recepies_url_list[:50]:
    print(url)
    content = str(requests.get(url).content)
    getName(content)
    getLevel(content)
    getNumberPeople(content)
    getIngredients(content)
    print()

https://www.cookingchanneltv.com/recipes/nigella-lawson/pasta-2203687
	Pasta
	Easy
	4
		pasta or Italian 00 flour-->Amount: 2(cups)
		Fat pinch salt-->Amount: ()
		Eggs-->Amount: 4()
		Meatballs-->Amount: ()
		Tomato Sauce-->Amount: ()

https://www.cookingchanneltv.com/recipes/chuck-hughes/pasta-al-forno-2194192
	Pasta Al Forno
	Easy
	4
		Salt-->Amount: ()
		rigatoni pasta-->Amount: 1(pound)
		fresh tomato sauce-->Amount: 8(ounces)
		fresh ricotta-->Amount: 4(ounces)
		chopped dried sopressata-->Amount: 4(ounces)
		meatballs-->Amount: 20(small)
		fresh mozzarella-->Amount: 4(ounces)
		hard-boiled eggs-->Amount: 4()
		Grated Parmigiano-Reggiano-->Amount: ()

https://www.cookingchanneltv.com/recipes/david-rocco/pasta-al-forno-oven-baked-pasta-1960559
	Pasta Al Forno
	Easy
	4
		rigatoni pasta-->Amount: 450(g)
		extra-virgin olive oil-->Amount: 60(ml)
		large eggplant-->Amount: 1()
		large sun-dried tomatoes-->Amount: 10()
		infornate olives-->Amount: 15()
		dried chile peppers-->Amount: 2

	Pasta Primavera
	Easy
	4
		olive oil-->Amount: 1(tablespoon)
		minced garlic-->Amount: 3(cloves)
		red bell pepper-->Amount: 1()
		thin asparagus-->Amount: 2(pound)
		sliced button mushrooms-->Amount: 1(cup)
		grape or cherry tomatoes-->Amount: 1(cup)
		low-sodium chicken stock-->Amount: 1(cup)
		1 percent milk-->Amount: 2(cup)
		all-purpose flour-->Amount: 1(tablespoon)
		salt-->Amount: 2(teaspoon)
		ground black pepper-->Amount: 2(teaspoon)
		large carrot peeled and sliced into strips with a peeler (about 2 cups carrot ribbons)-->Amount: 1()
		whole-wheat linguine-->Amount: 4(pound)
		grated Parmesan-->Amount: 2(cup)
		chopped parsley leaves-->Amount: 2(tablespoons)
		shredded basil leaves-->Amount: 4(cup)

https://www.cookingchanneltv.com/recipes/pasta-puttanesca-1945199
	Pasta Puttanesca
	Easy
	4
		whole-wheat thin spaghetti-->Amount: 8(ounces)
		extra-virgin olive oil-->Amount: 1(tablespoon)
		garlic-->Amount: 2(cloves)
		chopped flat-leaf parsley-->Amount: 3(cup)
		pitted choppe

	Pasta e Patate
	Easy
	4
		cup/50 ml extra-virgin olive oil-->Amount: 4()
		prosciutto or salami-->Amount: 100(g)
		white onion-->Amount: 1()
		Celery-->Amount: 1()
		Carrot-->Amount: 1()
		fresh chile pepper-->Amount: 1()
		Salt and freshly ground black pepper-->Amount: ()
		Potatoes-->Amount: 2()
		cups/600 ml water-->Amount: 2()
		tomato puree-->Amount: 300(g)
		spaghetti-->Amount: 300(g)
		cup/125 ml Pecorino cheese-->Amount: 2()
		cup/125 ml freshly grated cheese Parmigiano-->Amount: 2()

https://www.cookingchanneltv.com/recipes/pasta-with-anchovies-pasta-con-le-alice-1939572
	Pasta with Anchovies
	Easy
	6
		dry bucatoni-->Amount: 1(pound)
		heads fennel-->Amount: 2()
		sweet white onions-->Amount: 2(medium)
		olive oil-->Amount: 3(tablespoons)
		large anchovy fillets-->Amount: 15()
		saffron threads-->Amount: 1(tablespoon)
		currants-->Amount: 2(tablespoons)
		pine nuts-->Amount: 2(tablespoons)
		sugar-->Amount: 1(teaspoon)
		salt-->Amount: 1(teaspoon)
		pepper-->Amount: 4(teaspo

	Pastelles
	Advanced
	24
		precooked fine cornmeal-->Amount: 1(pound)
		blonde granulated coconut sugar-->Amount: 4(tablespoon)
		sea salt-->Amount: 2(tablespoon)
		diced white onion-->Amount: 2(cups)
		garlic-->Amount: 5(cloves)
		dried thyme-->Amount: 1(tablespoon)
		dried oregano-->Amount: 1(tablespoon)
		stick unsalted butter-->Amount: 1()

https://www.cookingchanneltv.com/recipes/pastelon-de-platano-maduro-2010425
	Pastelon de Platano Maduro
	None
	-1
		lean ground beef-->Amount: 1(pound)
		annatto infused oil or canola oil*-->Amount: 2(tablespoons)
		green bell pepper-->Amount: 1()
		aje dulce also known as ajicitos-->Amount: 3()
		white onion-->Amount: 1(medium)
		garlic-->Amount: 3(cloves)
		peeled whole tomatoes-->Amount: 1(cup)
		dry sherry-->Amount: 4(cup)
		raisins-->Amount: 1-2(tablespoon)
		whole bay leaf-->Amount: 1()
		kosher salt-->Amount: 1(teaspoon)
		dried oregano-->Amount: 1(teaspoon)
		freshly ground pepper-->Amount: 2(teaspoon)
		recao leaves-->Amount: 3()
		red 